In [5]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import xgboost as xgb
%matplotlib inline
import matplotlib
#matplotlib.use('agg')
matplotlib.style.use('ggplot')
import pickle as pkl
from matplotlib import pyplot as plt
from collections import Counter
from functools import reduce
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import random
random.seed(1991)

In [6]:
novel_compounds=pd.read_csv("/data/dharp/compounding/datasets/novel_compounds.txt",delim_whitespace=True,header=None)
novel_compounds.columns=['modifier','head']
modifiers_list=novel_compounds['modifier'].unique()
heads_list=novel_compounds['head'].unique()

In [7]:
dfm_dec=pd.read_csv("/data/dharp/compounding/datasets/dist_sem_dec.csv")
dfm_dec

modifier            head  1800_local_mi  1810_local_mi  \
0                   a_n         aaaaa_n          0.006          0.005   
1                   a_n       addison_n          0.006          0.005   
2                   a_n          anti_n          0.006          0.005   
3                   a_n       archive_n          0.006          0.005   
4                   a_n          atom_n          0.006          0.005   
5                   a_n             b_n          0.007          0.008   
6                   a_n          band_n          0.006          0.005   
7                   a_n      baseball_n          0.006          0.005   
8                   a_n      behavior_n          0.006          0.005   
9                   a_n          bell_n          0.006          0.005   
10                  a_n       berkley_n          0.006          0.005   
11                  a_n  bibliography_n          0.006          0.005   
12                  a_n          bill_n          0.006          0.005   
13                  a_n          bird_n          0.006          0.005   
14                  a_n         board_n          0.006          0.005   
15                  a_n          book_n          0.006          0.005   
16                  a_n       boswell_n          0.006          0.005   
17                  a_n         brand_n          0.006          0.005   
18                  a_n        briefe_n          0.006          0.005   
19                  a_n         bunch_n          0.006          0.005   
20                  a_n          burn_n          0.006          0.005   
21                  a_n             c_n          0.010          0.006   
22                  a_n      calendar_n          0.006          0.005   
23                  a_n          call_n          0.006          0.005   
24                  a_n      campaign_n          0.006          0.005   
25                  a_n       cassell_n          0.006          0.005   
26                  a_n           cat_n          0.006          0.005   
27                  a_n   cataloguing_n          0.006          0.005   
28                  a_n          cell_n          0.006          0.005   
29                  a_n       century_n          0.006          0.005   
...                 ...             ...            ...            ...   
775536           zuni_n      mountain_n          0.006          0.005   
775537           zuni_n        origin_n          0.006          0.005   
775538           zuni_n    prehistory_n          0.006          0.005   
775539           zuni_n      religion_n          0.006          0.005   
775540           zuni_n          ruin_n          0.006          0.005   
775541           zuni_n        sacred_n          0.006          0.005   
775542           zuni_n        social_n          0.006          0.005   
775543           zuni_n     tradition_n          0.006          0.005   
775544           zuni_n         tribe_n          0.006          0.005   
775545           zuni_n       village_n          0.006          0.005   
775546           zuni_n           war_n          0.006          0.005   
775547           zuni_n          word_n          0.006          0.005   
775548          zunyi_n    conference_n          0.006          0.005   
775549         zurich_n           acm_n          0.006          0.005   
775550         zurich_n     agreement_n          0.006          0.005   
775551         zurich_n       chamber_n          0.006          0.005   
775552         zurich_n    conference_n          0.006          0.005   
775553         zurich_n    government_n          0.006          0.005   
775554         zurich_n       seminar_n          0.006          0.005   
775555        zwemmer_n       gallery_n          0.006          0.005   
775556    zwinglische_n   reformation_n          0.006          0.005   
775557     zwitterion_n          form_n          0.006          0.005   
775558     zwitterion_n  intermediate_n          0.006          0.005   


In [8]:
all_compounds=dfm_dec[['modifier','head']]
all_compounds_list=list(zip(all_compounds.modifier.tolist(),all_compounds['head'].tolist()))
len(all_compounds_list)

775566

In [ ]:
modifiers=dfm_dec.loc[dfm_dec.modifier.isin(modifiers_list)].drop(['head','1800_sim_with_head','1810_sim_with_head',
'1820_sim_with_head','1830_sim_with_head','1840_sim_with_head','1850_sim_with_head','1860_sim_with_head','1870_sim_with_head',
 '1880_sim_with_head','1890_sim_with_head','1900_sim_with_head','1910_sim_with_head','1920_sim_with_head','1930_sim_with_head','1940_sim_with_head',
 '1950_sim_with_head','1960_sim_with_head','1970_sim_with_head','1980_sim_with_head','1990_sim_with_head'],axis=1)
modifier_features=modifiers.groupby(['modifier']).agg({'mean','std'})
modifier_features_1=modifier_features.columns.get_level_values(0)
modifier_features_2=modifier_features.columns.get_level_values(1)

cur_year=0
new_columns=[]
for year in modifier_features_1:
    new_columns.append("modifier_"+str(year)+"_"+modifier_features_2[cur_year])
    cur_year+=1
modifier_features.columns=new_columns
modifier_features.fillna(modifier_features.mean(),inplace=True)
modifier_features.reset_index(inplace=True)
modifier_features

modifier  modifier_1800_local_mi_std  \
0                  a_n                       0.001   
1                aaa_n                       0.000   
2             aarhus_n                       0.000   
3        abbreviated_n                       0.000   
4          abduction_n                       0.000   
5        abecedarian_n                       0.000   
6          abilities_n                       0.002   
7            ability_n                       0.000   
8      abnormalities_n                       0.000   
9          abolition_n                       0.000   
10     abolitionists_n                       0.000   
11        aboriginal_n                       0.000   
12        aborigines_n                       0.000   
13          abortion_n                       0.000   
14               abr_n                       0.000   
15            abrams_n                       0.000   
16            abroad_n                       0.000   
17           absence_n                       0.000   
18          absentee_n                       0.001   
19          absolute_n                       0.000   
20          absorber_n                       0.000   
21    absorptiometry_n                       0.000   
22        abstinence_n                       0.000   
23          abstract_n                       0.000   
24       abstraction_n                       0.000   
25         abstracts_n                       0.000   
26         abundance_n                       0.001   
27             abuse_n                       0.000   
28            abuses_n                       0.002   
29                ac_n                       0.000   
...                ...                         ...   
9049            year_n                       0.000   
9050        yearling_n                       0.000   
9051           years_n                       0.001   
9052           yeast_n                       0.000   
9053          yellow_n                       0.000   
9054             yes_n                       0.000   
9055       yesterday_n                       0.004   
9056           yield_n                       0.000   
9057            ymca_n                       0.000   
9058            yoga_n                       0.000   
9059            york_n                       0.000   
9060          yorker_n                       0.000   
9061          yoruba_n                       0.000   
9062           young_n                       0.000   
9063           youth_n                       0.001   
9064        yugoslav_n                       0.000   
9065               z_n                       0.000   
9066         zealand_n                       0.000   
9067         zedillo_n                       0.000   
9068             zen_n                       0.000   
9069         zeolite_n                       0.000   
9070            zero_n                       0.000   
9071      zidovudine_n                       0.000   
9072        zimbabwe_n                       0.000   
9073            zion_n                       0.000   
9074         zionist_n                       0.000   
9075             zip_n                       0.000   
9076           zonal_n                       0.000   
9077            zone_n                       0.000   
9078            zuni_n                       0.000   

      modifier_1800_local_mi_mean  modifier_1810_local_mi_std  \
0                           0.006                       0.000   
1                           0.006                       0.000   
2                           0.006                       0.000   
3                           0.006                       0.000   
4                           0.006                       0.000   
5                           0.006                       0.000   
6                           0.006                       0.001   
7                           0.006                       0.000   
8                           0.006                       0.000   
9              

In [ ]:
heads=dfm_dec.loc[dfm_dec['head'].isin(heads_list)].drop(['modifier','1800_sim_with_modifier', '1810_sim_with_modifier',
 '1820_sim_with_modifier','1830_sim_with_modifier','1840_sim_with_modifier','1850_sim_with_modifier','1860_sim_with_modifier',
 '1870_sim_with_modifier','1880_sim_with_modifier','1890_sim_with_modifier','1900_sim_with_modifier','1910_sim_with_modifier',
 '1920_sim_with_modifier','1930_sim_with_modifier','1940_sim_with_modifier','1950_sim_with_modifier','1960_sim_with_modifier',
 '1970_sim_with_modifier','1980_sim_with_modifier','1990_sim_with_modifier'],axis=1)
head_features=heads.groupby(['head']).agg({'mean','std'})
head_features_1=head_features.columns.get_level_values(0)
head_features_2=head_features.columns.get_level_values(1)

cur_year=0
new_columns=[]
for year in head_features_1:
    new_columns.append("head_"+str(year)+"_"+head_features_2[cur_year])
    cur_year+=1
head_features.columns=new_columns
head_features.fillna(head_features.mean(),inplace=True)
head_features.reset_index(inplace=True)
head_features

head  head_1800_local_mi_std  head_1800_local_mi_mean  \
0                a_n                   0.001                    0.006   
1            aaron_n                   0.000                    0.006   
2               ab_n                   0.001                    0.006   
3      abandonment_n                   0.000                    0.006   
4     abbreviation_n                   0.000                    0.006   
5              abc_n                   0.000                    0.006   
6        abduction_n                   0.000                    0.006   
7            abdul_n                   0.000                    0.006   
8         abdullah_n                   0.000                    0.006   
9          ability_n                   0.000                    0.006   
10      abjuration_n                   0.000                    0.006   
11        ablation_n                   0.000                    0.006   
12     abnormality_n                   0.000                    0.006   
13           abode_n                   0.000                    0.006   
14    abolitionist_n                   0.000                    0.006   
15          abound_n                   0.000                    0.006   
16         abounds_n                   0.000                    0.006   
17         abraham_n                   0.002                    0.006   
18      abridgment_n                   0.001                    0.006   
19          abroad_n                   0.000                    0.006   
20        absentee_n                   0.000                    0.006   
21        absolute_n                   0.000                    0.006   
22      absolutism_n                   0.000                    0.006   
23      absorption_n                   0.000                    0.006   
24      abstinence_n                   0.000                    0.006   
25        abstract_n                   0.001                    0.006   
26     abstraction_n                   0.000                    0.006   
27             abu_n                   0.000                    0.006   
28       abundance_n                   0.000                    0.006   
29           abuse_n                   0.000                    0.006   
...              ...                     ...                      ...   
7697          yard_n                   0.001                    0.006   
7698          yawn_n                   0.000                    0.006   
7699          year_n                   0.002                    0.006   
7700      yearning_n                   0.000                    0.006   
7701         yeast_n                   0.000                    0.006   
7702          yell_n                   0.000                    0.006   
7703         yemen_n                   0.000                    0.006   
7704           yes_n                   0.000                    0.006   
7705     yesterday_n                   0.001                    0.006   
7706         yield_n                   0.000                    0.006   
7707          yoga_n                   0.000                    0.006   
7708        yonder_n                   0.000                    0.006   
7709          york_n                   0.000                    0.006   
7710        yoruba_n                   0.000                    0.006   
7711         young_n                   0.000                    0.006   
7712         yours_n                   0.000                    0.006   
7713         youth_n                   0.000                    0.006   
7714      zanzibar_n                   0.000                    0.006   
7715      zealotry_n                   0.000                    0.006   
7716     zechariah_n                   0.000                    0.006   
7717          zero_n                   0.000                    0.006   
7718          zeus_n                   0.000                    0.006   
7719         zheng_n                   0.000                    0.006   
7

## Positive Set

In [ ]:
novel_df=pd.merge(novel_compounds,head_features,on=["head"])
novel_df=pd.merge(novel_df,modifier_features,on=["modifier"])
novel_df['Plausibility']=True
novel_df

modifier                head  head_1800_local_mi_std  \
0                a_n          peaceful_n                   0.000   
1                a_n          robinson_n                   0.000   
2                a_n            supply_n                   0.000   
3            india_n          peaceful_n                   0.000   
4            india_n          standard_n                   0.000   
5            india_n             award_n                   0.000   
6            india_n             joint_n                   0.001   
7            india_n         companion_n                   0.001   
8            india_n        foundation_n                   0.000   
9            india_n         gentleman_n                   0.010   
10           india_n              boat_n                   0.000   
11           india_n               pay_n                   0.000   
12           india_n              wife_n                   0.003   
13           india_n           othello_n                   0.000   
14           india_n              sage_n                   0.000   
15             the_n          robinson_n                   0.000   
16             the_n          evidence_n                   0.003   
17             the_n       association_n                   0.000   
18             the_n             crane_n                   0.000   
19             the_n             chase_n                   0.000   
20             the_n              amex_n                   0.000   
21             the_n               fox_n                   0.000   
22             the_n         listening_n                   0.000   
23            book_n            supply_n                   0.000   
24            book_n            record_n                   0.001   
25            book_n              link_n                   0.000   
26            book_n      subscription_n                   0.000   
27            book_n          molecule_n                   0.000   
28            book_n     globalization_n                   0.000   
29            book_n               the_n                   0.000   
...              ...                 ...                     ...   
28418     vengeful_n             groom_n                   0.000   
28419        venom_n               mud_n                   0.000   
28420         verb_n            plural_n                   0.000   
28421    verdigris_n            copper_n                   0.000   
28422       verlag_n            italia_n                   0.000   
28423          via_n             carlo_n                   0.000   
28424        vigil_n           walking_n                   0.000   
28425      vitriol_n            splash_n                   0.000   
28426         vivo_n  characterisation_n                   0.000   
28427    volcanoes_n              etna_n                   0.000   
28428    voluntary_n     interindustry_n                   0.000   
28429    voluntary_n       theological_n                   0.000   
28430       waiver_n            entail_n                   0.000   
28431      wasting_n       distraction_n                   0.001   
28432   waterhouse_n           hawkins_n                   0.000   
28433        weave_n              eden_n                   0.001   
28434         well_n             timon_n                   0.000   
28435          wet_n       granulation_n                   0.000   
28436      whaling_n             haven_n                   0.004   
28437  whitechapel_n              slum_n                   0.000   
28438      whitsun_n        bridegroom_n                   0.000   
28439      wildcat_n             swamp_n                   0.002   
28440     windlass_n            chorus_n                   0.000   
28441       wooden_n               ark_n                   0.000   
28442       wooden_n         spectacle_n                   0.001   
28443  woodlanders_n         desperate_n                   0.000   
28444       worthy_n            freind_n                   0.000   


In [ ]:
available_novel_compounds=novel_df[['modifier','head']]
available_novel_compounds_list=list(zip(available_novel_compounds.modifier.tolist(),available_novel_compounds['head'].tolist()))
available_modifiers=available_novel_compounds.modifier.unique().tolist()
available_heads=available_novel_compounds['head'].unique().tolist()
len(available_novel_compounds_list)

28448

## Creation of Negative Classes - CorruptHead and CorruptModifier

In [ ]:
len(all_compounds_list)

775566

In [ ]:
i=0
corrupt_modifier_list=[]
corrupt_head_list=[]
for cur_num,(modifier,head) in enumerate(available_novel_compounds_list):
    # CorruptHead , Modifier is same
    #print(modifier,head)
    if cur_num%1000==0:
        print(cur_num)
    while True:
        temp_corrupt_head=random.choice(available_heads)
        if (modifier,temp_corrupt_head) not in available_novel_compounds_list:
            if (modifier,temp_corrupt_head) not in all_compounds_list:
                corrupt_head_list.append((modifier,temp_corrupt_head))
                break
    # CorruptModifier , Head is same
    while True:
        temp_corrupt_modifier=random.choice(available_modifiers)
        if (temp_corrupt_modifier,head) not in available_novel_compounds_list:
            if (temp_corrupt_modifier,head) not in all_compounds_list:
                corrupt_modifier_list.append((temp_corrupt_modifier,head))
                break
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [ ]:
corrupt_head=pd.DataFrame(corrupt_head_list, columns=['modifier', 'head'])
corrupt_head=pd.merge(corrupt_head,head_features,on=["head"])
corrupt_head=pd.merge(corrupt_head,modifier_features,on=["modifier"])
#final_df=pd.merge(final_df_1,final_df_1,on=['modifier','head'])
corrupt_head['Plausibility']=False
corrupt_head=pd.concat([novel_df.set_index(['modifier','head']),corrupt_head.set_index(['modifier','head'])],sort=False)
corrupt_head

## XGBoost

In [45]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [62]:
xgb_model = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=10)

In [63]:
X, Y = corrupt_head.iloc[:,:-1],corrupt_head.iloc[:,-1]

In [64]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [65]:
folds = 10
param_comb = 10

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb_model, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=10, cv=skf.split(X,Y), verbose=3, random_state=1991 )

In [ ]:
random_search.fit(X, Y)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:  7.2min


In [38]:
xg_model.fit(X_train,y_train)

XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [39]:
preds = xg_model.predict(X_test)

In [41]:
pd.crosstab(preds,y_test)

Plausibility  False  True 
row_0                     
False          3009   2443
True           2629   3299